In [1]:
import tensorflow as tf
import numpy as np
import cv2
import tqdm
import os, sys
import matplotlib.pyplot as plt
import random

In [2]:
def image_load_and_batch(img_list, sizing):
    im = []
    labels = []
    
    for batch in img_list:
        
        img_name, label = batch
        img =cv2.resize(cv2.imread(img_name), (sizing,sizing))
        b, g, r = cv2.split(img)
        im.append(cv2.merge([r,g,b]))
        if label == 1:
            labels.append([0.,1.])
        if label ==0:
            labels.append([1.,0.])
        

        #print(np.shape(im),np.shape(labels))
    return im, labels

In [3]:
def dataLoad():
    
    print("Data Load Starting..!")
    path = "../SPORT_QA/DataSet(190408)/"
    path = [os.path.join(path, x) for x in os.listdir(path)] 
    path2 = [os.path.join(x, y) for x in path if os.path.isdir(x) for y in os.listdir(x)]

    path3= [os.path.join(x, y) for x in path2 if x[-8:] =="box_crop" for y in os.listdir(x)]

    path2 = [os.path.join(x, y) for x in path3 if os.path.isdir(x) for y in os.listdir(x)]
    path3 = [os.path.join(x, y) for x in path2 if os.path.isdir(x) for y in os.listdir(x)]
    Ball_path= [[os.path.join(x, y), 1] for x in path3 if os.path.isdir(x) for y in os.listdir(x) if not x[-6:]=="NoBall"]
    NBall_path = [[os.path.join(x, y), 0] for x in path3 if os.path.isdir(x) for y in os.listdir(x) if x[-6:]=="NoBall"]
    
    Train_data =  Ball_path[:(len(Ball_path)//4) * 3] + NBall_path[:(len(NBall_path)//4) * 3]
    Test_data = NBall_path[(len(NBall_path)//4) * 3:] + Ball_path[(len(Ball_path)//4) * 3:]
    
    print("DataLoad Done..")
    print("Train data.. " , len(Train_data), "Ball : ", len(Ball_path[:(len(Ball_path)//4) * 3]), " NBall : ", len(NBall_path[:(len(NBall_path)//4) * 3]))
    print("Test data.. " , len(Test_data),  "Ball : ", len(Ball_path[(len(Ball_path)//4) * 3:]), " NBall : " ,len(NBall_path[(len(NBall_path)//4) * 3:]))
    return Train_data, Test_data


In [4]:
def block(sess, data, filters, stride,  out_ch):
   
    
    conv = tf.layers.conv2d(inputs = data, filters=  out_ch, kernel_size=filters, strides=stride, padding="same")
    bn = tf.contrib.layers.batch_norm(inputs = conv,  scale = False,updates_collections=None)
    relu = tf.nn.relu(bn)
    maxpool = tf.layers.max_pooling2d(inputs= conv,  pool_size= 2, strides=2)
    
    return maxpool

In [5]:
def block_me(sess, data, filters1, stride1, filters2, stride2,  out_ch1, out_ch2):

    
    conv = tf.layers.conv2d(inputs = data, filters=  out_ch1, kernel_size=filters1, strides=stride1, padding="same")
    conv2 = tf.layers.conv2d(inputs = conv, filters=  out_ch2, kernel_size=filters2, strides=stride2, padding="same")
    
    return conv2

In [6]:
def Upsample_block(data, output_size, kernel=2, std=2):
    
    output = tf.layers.conv2d_transpose(data, filters=output_size, kernel_size=kernel, strides=std, padding="VALID")
    
    return output

In [10]:
def Model(sess, data, labels, lr):
    
    with tf.variable_scope("block1"):
        block1 = block(sess, data,filters=5, stride=1, out_ch=64)
        print("Load Success block1")
        
    with tf.variable_scope("block2"):
        block2 = block(sess, block1,filters=5, stride=1, out_ch = 64)
        print("Load Success block2")
        
    with tf.variable_scope("block3"):
        block3 = block(sess, block2,filters=5, stride=1, out_ch =64)
        print("Load Success block3")
        
    with tf.variable_scope("block4"):
        block4 = block(sess, block3,filters=5, stride=1, out_ch = 64)
        print("Load Success block4")
        
    return block3

In [9]:
def Model2(sess, data, labels, lr):
    
    with tf.variable_scope("block12"):
        block1 =block_me(sess, data,filters1=7, stride1=2, filters2=5, stride2=1, out_ch1 = 8, out_ch2 = 8)
        print(np.shape(block1))
        print("Load Success block12")
        
    with tf.variable_scope("block22"):
        block2 = block_me(sess, block1,filters1=5, stride1=1, filters2=3, stride2=1, out_ch1 = 16, out_ch2 = 16)
        block2 = tf.layers.max_pooling2d(inputs= block2,  pool_size= 2, strides=2)
        print(np.shape(block2))
        print("Load Success block22")

        
    with tf.variable_scope("block32"):
        block3 = block_me(sess,block2,filters1=3, stride1=1, filters2=3, stride2=1, out_ch1 = 32, out_ch2 =32)
        block3 = tf.layers.max_pooling2d(inputs= block3,  pool_size= 2, strides=2)
        print(np.shape(block3))
        print("Load Success block32")
        
    upsize = block1.get_shape().as_list()[-1] 
    with tf.variable_scope("Upsize1"):
        block2_up = Upsample_block(block2 , upsize, kernel=2, std=2) 
        print(np.shape(block2_up))
        print("Load Success block32")
        
    with tf.variable_scope("Upsize2"):
        block3_up = Upsample_block(block3, upsize, kernel=4, std=4)
        print(np.shape(block3_up))
        print("Load Success block32")
        
        
    with tf.variable_scope("Concat_conv"):
        Conv_concat = tf.concat([block1,block2_up, block3_up],axis=3)  
        print("Concat Done")

        
    with tf.variable_scope("block42"):
        block4 = block_me(sess,Conv_concat,filters1=3, stride1=1, filters2=3, stride2=1, out_ch1 = 2, out_ch2 = 2)
        #block4 = tf.layers.max_pooling2d(inputs= block4,  pool_size= 2, strides=2)
        print(np.shape(block4))
        print("Load Success block42")
        
    return block4

In [7]:
def optimizer(data, labels, lr):
    #print(tf.size(labels))
    #data = tf.reshape(data, [None,-1])
    data = tf.layers.flatten(data)
    layer1 = tf.layers.dense(data, 4096)
    layer2 = tf.layers.dense(layer1, 4096)
    layer = tf.layers.dense(layer2, 2)
    logits = tf.nn.softmax(layer)
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer, labels=label)
    loss_op = tf.reduce_mean(cross_entropy, name = "loss")
    trian_op = tf.train.GradientDescentOptimizer(learning_rate = lr).minimize(loss_op, name="train_op")
    
    return logits, loss_op, trian_op

In [8]:
train_data, test_data = dataLoad()
len(test_data)

Data Load Starting..!
DataLoad Done..
Train data..  96927 Ball :  24051  NBall :  72876
Test data..  32312 Ball :  8020  NBall :  24292


32312

In [11]:
n_epoch=200
batch_size = 32
train_data, test_data = dataLoad()
lr = 0.00001
tf.reset_default_graph()
sizing = 64


input_image = tf.placeholder(tf.float32,[None, sizing,sizing,3])
label = tf.placeholder(tf.float32,[None,2])
init = tf.global_variables_initializer()

with tf.Session(config=tf.ConfigProto(gpu_options=(tf.GPUOptions(per_process_gpu_memory_fraction=0.7)), log_device_placement=True)) as sess:
    
    output = Model2(sess, input_image, label, lr)
    logits, losses, train_op = optimizer(output, label, lr)
    
    saver = tf.train.Saver(max_to_keep = 8000000)
    
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    
    
    
    for epoch in range(n_epoch):
        all_loss = 0
        all_acc = 0
        batch = 0

        random.shuffle(train_data)
        print("\n\nTraining Start.. ")
        count = 0
#         training = tf.constant(True)
       
        while True:
            

            now_img, now_gt = image_load_and_batch(train_data[batch : batch + batch_size], sizing)
            #print(batch , batch +batch_size)
            '''except cv2.error:
                continue'''
            logit, loss, _ = sess.run([logits, losses, train_op], feed_dict={input_image:now_img, label:now_gt})

            pred = np.argmax(logit, axis=1)
            gt_label = np.argmax(now_gt, axis=1)
            acc =(1 - (len(np.nonzero( pred- gt_label)[0]) /  len(gt_label))) * 100
            all_loss += loss
            all_acc += acc
            batch += batch_size
            #print("Now..", batch,"~",batch+batch_size, " Acc : ", acc)
            count += 1
            if batch > 96896: break
            

        print(epoch," epoch is end..")
        print("Now Loss : ", all_loss, " Now Acc : ", all_acc / count)

        print("Testing Start..")

        all_loss = 0
        all_acc = 0
        random.shuffle(test_data)
        batch = 0
        count = 0

        while True:

            now_img, now_gt = image_load_and_batch(test_data[batch : batch + batch_size] , sizing)
            test_logit, test_loss, _ = sess.run([logits, losses, train_op], feed_dict={input_image:now_img, label:now_gt})

            pred = np.argmax(test_logit, axis=1)
            gt_label = np.argmax(now_gt, axis=1)
            acc =(1 - (len(np.nonzero( pred- gt_label)[0]) /  len(gt_label))) * 100
            all_loss += test_loss
            all_acc += acc
            batch += batch_size
            count += 1
            if batch > 32288: break

            
        print("Testing is end..")
        print("Now Loss : ", all_loss, " Now Acc : ", all_acc / count)

        saver.save(sess, './Result/Ball_model_' + str(epoch)+ '.ckpt')

Data Load Starting..!
DataLoad Done..
Train data..  96927 Ball :  24051  NBall :  72876
Test data..  32312 Ball :  8020  NBall :  24292
(?, 32, 32, 8)
Load Success block12
(?, 16, 16, 16)
Load Success block22
(?, 8, 8, 32)
Load Success block32
(?, 32, 32, 8)
Load Success block32
(?, 32, 32, 8)
Load Success block32
Concat Done
(?, 32, 32, 2)
Load Success block42
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



Training Start.. 
0  epoch is end..
Now Loss :  2955.9937513694167  Now Acc :  82.24751461676908
Testing Start..
Testing is end..
Now Loss :  543.5548333823681  Now Acc :  86.2809405940594


Training Start.. 
1  epoch is end..
Now Loss :  1400.2216491475701  Now Acc :  86.02757350983504
Testing Start..
Testing is end..
Now Loss :  432.1023721024394  Now Acc :  87.77227722772277


Training Start.. 
2  epoch is end..
Now Loss :  1239.267601

Now Loss :  154.50583614595234  Now Acc :  94.24298679867988


Training Start.. 
39  epoch is end..
Now Loss :  539.3996095154434  Now Acc :  93.13095054260428
Testing Start..
Testing is end..
Now Loss :  154.2607901878655  Now Acc :  94.29764851485149


Training Start.. 
40  epoch is end..
Now Loss :  539.6130309663713  Now Acc :  93.126757207212
Testing Start..
Testing is end..
Now Loss :  151.48504566960037  Now Acc :  94.3667491749175


Training Start.. 
41  epoch is end..
Now Loss :  536.2551862038672  Now Acc :  93.12679048765163
Testing Start..
Testing is end..
Now Loss :  151.60324551071972  Now Acc :  94.28733498349834


Training Start.. 
42  epoch is end..
Now Loss :  532.9633134845644  Now Acc :  93.1742816749912
Testing Start..
Testing is end..
Now Loss :  151.06190377101302  Now Acc :  94.36468646864687


Training Start.. 
43  epoch is end..
Now Loss :  530.2116465326399  Now Acc :  93.23411990543029
Testing Start..
Testing is end..
Now Loss :  148.6346229761839  Now Acc :

KeyboardInterrupt: 

In [ ]:
test_data